In [4]:
import os

In [5]:
%pwd

'/Users/venkyroyal/nandu_practise/TextSummarization/research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'/Users/venkyroyal/nandu_practise/TextSummarization'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [18]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer
          params = self.params.TrainingArguments

          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
                root_dir= config.root_dir,
                data_path= config.data_path,
                model_ckpt= config.model_ckpt,
                num_train_epochs= params.num_train_epochs,
                warmup_steps = params.warmup_steps,
                per_device_train_batch_size= params.per_device_train_batch_size,
                weight_decay= params.weight_decay,
                logging_steps= params.logging_steps,
                evaluation_strategy= params.evaluation_strategy,
                eval_steps= params.eval_steps,
                save_steps= params.save_steps,
                gradient_accumulation_steps= params.gradient_accumulation_steps

          )

          return model_trainer_config

In [20]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [21]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir= self.config.root_dir, num_train_epochs = self.config.num_train_epochs, warmup_steps= self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size= self.config.per_device_train_batch_size,
            weight_decay= self.config.weight_decay, logging_steps= self.config.logging_steps,
            evaluation_strategy= self.config.evaluation_strategy, eval_steps= self.config.eval_steps, save_steps= 1e6,
            gradient_accumulation_steps= self.config.gradient_accumulation_steps
        )

        trainer = Trainer(model = model_pegasus, args = trainer_args,
                          tokenizer= tokenizer, data_collator= seq2seq_data_collator,
                          train_dataset= dataset_samsum_pt['test'],
                          eval_dataset= dataset_samsum_pt['validation'])
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [23]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()

except Exception as e:
    raise e

[2024-01-19 19:07:46,592: INFO: yaml file: config/config.yaml loaded successfully]
[2024-01-19 19:07:46,607: INFO: yaml file: params.yaml loaded successfully]
[2024-01-19 19:07:46,608: INFO: created directory at: artifacts]
[2024-01-19 19:07:46,612: INFO: created directory at: artifacts/model_trainer]


  0%|          | 0/51 [122:20:38<?, ?it/s]
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: MPS backend out of memory (MPS allocated: 6.87 GB, other allocations: 2.00 GB, max allowed: 9.07 GB). Tried to allocate 375.40 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).